# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [7]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 29 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 2s (512 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubunt

### Importing the libraries

In [4]:
import os
import random
import numpy as np
import torch
import torch.nn as nn #neural network module
import torch.optim as optim #optimization
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

In [5]:
class Network(nn.Module):

    def __init__(self, state_sie, action_size, seed = 42):
        super(Network, self).__init__()

        # This line calls the constructor of the parent class (nn.Module). It initializes the basic functionalities of the neural network module.
        self.seed = torch.manual_seed(seed)

        # This line defines the first fully connected (linear) layer of the network. It takes the state_size as input and outputs 64 values.
        self.fcl = nn.Linear(state_size, 64)

        # This line defines the second fully connected layer. It takes the 64 outputs from the first layer as input and outputs another 64 values.
        self.fc2 = nn.Linear(64, 64)

        # This line defines the third and final fully connected layer. It takes the 64 outputs from the second layer as input and outputs action_size values, corresponding to the possible actions.
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, state):

        # These two lines propagate the singal from the input layer to the first fully connected layer with a rectifier activation function
        x = self.fcl(state)
        x = F.relu(x) # rectifier activation function

        # These two lines propagate the first layer to second layer
        x = self.fc2(x)
        x = F.relu(x)

        # This line is to finally return the final signal in the outpu layer containing our actions
        x = self.fc3(x)
        return x



### Creating the architecture of the Neural Network

## Part 2 - Training the AI

### Setting up the environment

In [8]:
import gymnasium as gym
env = gym.make('LunarLander-v3') # The Lunar Lander environment was upgraded to v3
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


### Initializing the hyperparameters

In [9]:
learning_rate = 5e-4 # the best value after experiment

minibatch_size = 100

# the discount factor represents the present value of future rewards
# if we have a low discount factor -> 0, that make the agent shortsighted, only focus on current rewards
# if we have a high discount factor -> 1, that make the agent consider the future rewards in its accumulation of the total reward
discount_factor = 0.99

replay_buffer_size = int(1e5)  # the memory of AI, set up as 100k

interpolation_parameter = 1e-3


### Implementing Experience Replay

In [10]:
class ReplayMemory(object):

  def __init__(self, capacity): # the initial of capacity is replay_buffer_size
      self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # if we want to execute the code with faster and more performance
      self.capacity = capacity
      self.memory = [] # the list we store the experience

  def push(self, event):
      self.memory.append(event)
      # remove the oldest memory if the memory size is over than the capacity
      if len(self.memory) > self.capacity:
          del self.memory[0]

  def sample(self, batch_size):
      experiences = random.sample(self.memory, k = batch_size)
      states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
      actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
      rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
      next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
      dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device) # set the type as boolean type
      return states, next_states, actions, rewards, dones


### Implementing the DQN class

In [17]:
# crate the agent class to define the behavior of an agent that interact with the environment using a deep Q network.
# and while it's interacting with this environment, the agent maintains two Q networks
# first, the local Q network and then the target Q network
# the local Q network that will select the actions and target Q network will calculate the target Q values that will be used in training of the local Q network
# the soft update method will update the target Q network parameters by blending them with those of the local Q network -- in order to prevent abrupt changes which could destabilize the training
# the act method will help the agent choose an action based on its current understanding of the optimal policy, those acions will be returned from local Q network, that will forward propagate the state to return the action values
# the learn method use experiences that are sampled from the replay memory in order to update the local Q network Q values towards the target Q values



class Agent():

  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # if we want to execute the code with faster and more performance
    self.state_size = state_size
    self.action_size = action_size

    # Create two Q network
    self.local_qnetwork = Network(state_size, action_size).to(self.device)
    self.target_qnetwork = Network(state_size, action_size).to(self.device)

    # Build optimizer
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)

    # Memory of AI
    self.memory = ReplayMemory(replay_buffer_size)

    # Time step counter to decide in which moment to learn and update the network parameters
    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1) % 4  # reset every four steps
    if self.t_step == 0:  # learning every four steps
      if len(self.memory.memory) > minibatch_size: # we can learn when we have 100 observations
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)

  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad(): # make sure that every gradient computation is disabled
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()

    # eplison greedy selections
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, gamma):
    state, next_state, actions, rewards, dones = experiences # unstack
    next_q_targets = self.target_qnetwork(next_state).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + (discount_factor * next_q_targets * (1 - dones))
    q_expected = self.local_qnetwork(state).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward() # back propogate the loss
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

### Initializing the DQN agent

In [18]:
agent = Agent(state_size, number_actions)

### Training the DQN agent

In [19]:
number_episodes= 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value = 1.0 # 探索率
epsilon_ending_value = 0.01 # 最小探索率
epsilon_decay_rate = 0.995 # 探索率衰减
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_rate * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end="") # \r dynamic printing
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), "checkpoint.pth")
    break

Episode 100	Average Score: -166.23
Episode 200	Average Score: -117.79
Episode 300	Average Score: -43.59
Episode 400	Average Score: -8.02
Episode 500	Average Score: 84.02
Episode 600	Average Score: 187.54
Episode 634	Average Score: 200.21
Environment solved in 534 episodes!	Average Score: 200.21


## Part 3 - Visualizing the results

In [20]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()